In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import geocoder
from joblib import dump, load
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from pandas_profiling import ProfileReport


"""
Data
"""
data_info = pd.read_csv('hotels_information.csv')
print("data_info = ",data_info.shape)
data_pricing = pd.read_csv('pricing_data.csv')
print("data_pricing = ",data_pricing.shape)

"""
Info. table
"""
def info_table(df):
    tb = pd.DataFrame(df.dtypes)
    tb.columns = ['Type']
        
    num_unique = []
    for i in list(tb.index):
        num_unique.append(len(df[i].unique()))
            
    max_value = []
    min_value = []
    mean_value = []
    std_value = []
    mode_value = []
    for i in list(tb.index):
        if tb.loc[i,'Type'] == 'int64':
            max_value.append(round(df[i].max(),3))
            min_value.append(round(df[i].min(),3))
            mean_value.append(round(df[i].mean(),3))
            std_value.append(round(df.loc[:,i].std(),3))
            mode_value.append(df[i].mode(dropna=True)[0])
        elif tb.loc[i,'Type'] == 'float64':
            max_value.append(round(df[i].max(),3))
            min_value.append(round(df[i].min(),3))
            mean_value.append(round(df[i].mean(),3))
            std_value.append(round(df.loc[:,i].std(),3))
            mode_value.append(df[i].mode(dropna=True)[0])
        else:
            max_value.append('na')
            min_value.append('na')
            mean_value.append('na')
            std_value.append('na')
            mode_value.append(df[i].mode(dropna=True)[0])
        
    tb['Min'] = min_value
    tb['Max'] = max_value
    tb['Mean'] = mean_value
    tb['STD'] = std_value
    tb['Unique'] = num_unique
    tb['Zero Values'] = (df == 0.00).astype(int).sum(axis=0)
    tb['Missing Values'] = df.isnull().sum()
    tb['% Missing Values'] = round(100 * df.isnull().sum() / len(df),3)
    tb['mode'] = mode_value
    tb = tb.sort_values('Type')        
    return tb;

"""
Country label
"""
def country(lat,long):
    g = geocoder.osm([lat, long], method='reverse',short_name=False)
#     try:
#         get = g.json['country_code']
#     except KeyError:
#         get = g.json['country_code']    
    return g.country_code;   

locations = list(zip(data_info['latitude'], data_info['longitude']))

# country_list = []
# for index, tuple in enumerate(locations):
#     country_list.append(country(tuple[0], tuple[1]))

# dump(country_list,'country_list.sav')
country_list = load('country_list.sav')
data_info = pd.concat([data_info,pd.DataFrame(country_list,columns=['country'])],axis = 1)

"""
Remove missing/transform observations 
"""
# data_info_no_na = data_info.dropna()
# data_pricing_no_na = data_pricing.dropna()
# data_info_no_na["stars"] = data_info_no_na["stars"].astype("int")
# data_info_no_na["stars"] = data_info_no_na["stars"].astype("object")
# data_info = data_info.fillna(0)
# print("data_info_no_na = ",data_info_no_na.shape)
# print("data_pricing_no_na = ",data_pricing_no_na.shape)

"""
DF
"""
df = data_info.merge(data_pricing, how = 'inner',on ='our_hotel_id')
df['is_sold_out'] = np.where(df['is_sold_out'] == True,1,0)
df['arrival_date']= pd.to_datetime(df['arrival_date'],format='%Y/%m/%d').dt.normalize()
print('df = ',df.shape)

In [ ]:
"""
Simple 
"""
# ax = sns.boxplot(x="country", y="room_count", data=df)
# ax = sns.barplot(x="country", y="price_value_ref", data=df)

In [ ]:
"""
Complex
"""
# ax = sns.boxplot(x="stars", y="room_count",hue="country", data=data_info_no_na)
# ax = sns.barplot(x="meal_type_included", y="is_sold_out",hue="country", data=df)
# sns.scatterplot(data=df, x="lead_time", y="is_sold_out", hue="country");

In [ ]:
"""
Correlations
"""
# corr = data_pricing_no_na[['max_persons','price_value_ref','price_value_non_ref']].corr()# plot the heatmap
# ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))
# ax.set_yticklabels(
#     ax.get_yticklabels(),
#     rotation=0
# )

# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

### Q. 2.1.

In [ ]:
latitude = 50.855711
longitude = 4.359679

icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(20, 20)
    });
}"""

map_eu = folium.Map(location=[latitude, longitude], zoom_start=7,tiles='cartodbpositron')

locations = list(zip(data_info['latitude'], data_info['longitude']))
popups = ["lon:{}<br>lat:{}".format(lon, lat) for (lat, lon) in locations]


marker_cluster = MarkerCluster(
    locations=locations,
    popups=popups,
    name="1000 clustered icons",
    overlay=True,
    control=True,
    icon_create_function=icon_create_function,
)

marker_cluster.add_to(map_eu)

folium.LayerControl().add_to(map_eu)

map_eu

### Q. 2.2.

In [ ]:
df_be = df[df['country']== 'be']
df_nl = df[df['country']== 'nl']
print(df_be.shape)
print(df_nl.shape)

#### Q.2.2.1.

In [ ]:
def availability(df):
    df_cap = df.pivot_table(index='arrival_date', values= 'is_sold_out', aggfunc='sum')
    df_cap['Available'] = len(df['our_hotel_id'].unique()) - df_cap['is_sold_out']
    return df_cap;

sns.set(rc={'figure.figsize':(16,8.27),'figure.facecolor':'white'})
ax = sns.lineplot(data=availability(df), x=availability(df).index, y="Available",color='#6C5B7B',label="All")
sns.lineplot(data = availability(df_be), x=availability(df_be).index, y="Available",color='#355C7D',label="BE")
sns.lineplot(data = availability(df_nl), x=availability(df_nl).index, y="Available",color='#F8B195',label="NL")
ax.set(title= 'Availability')
ax.tick_params(labelsize=10)
plt.xticks(availability(df).index,rotation=90);

NL

2 apr	vrijdag	Good Friday	National holiday
4 apr	zondag	Easter Sunday	National holiday
5 apr	maandag	Easter Monday	National holiday

Dutch Flower Parade 2021, April 17, 2021 in Amsterdam

#### Q.2.2.2.

In [ ]:
def mean_price(df,type_price):
    df_pr = df.pivot_table(index='arrival_date', values= type_price, aggfunc='mean')
    return df_pr;

sns.set(rc={'figure.figsize':(16,8.27),'figure.facecolor':'white'})
ax = sns.lineplot(data=mean_price(df,'price_value_ref'), x=mean_price(df,'price_value_ref').index, y="price_value_ref",color='#6C5B7B',label="All")
sns.lineplot(data = mean_price(df_be,'price_value_ref'), x=mean_price(df_be,'price_value_ref').index, y="price_value_ref",color='#355C7D',label="BE")
sns.lineplot(data = mean_price(df_nl,'price_value_ref'), x=mean_price(df_nl,'price_value_ref').index, y="price_value_ref",color='#F8B195',label="NL")
ax.set(title= 'Average price')
ax.tick_params(labelsize=10)
plt.xticks(mean_price(df_nl,'price_value_ref').index,rotation=90);
    

#### Q. 2.2.3.

In [ ]:
ax = sns.distplot(df_be['review_score'],color='#355C7D',label="BE")
sns.distplot(df_nl['review_score'],color='#F8B195',label="NL")
ax.set_xticks(range(-1,10))
ax.legend(labels=['BE','NL']);

#### Q.2.2.4.

In [ ]:
df_pred = df.pivot_table(index='name', values= 'is_sold_out', aggfunc='sum')
df_pred['is_sold_out'] = np.where(df_pred['is_sold_out'] > 0,1,0)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'price_value_ref', aggfunc='mean'),how = 'inner',left_index=True, right_index=True)
# df_pred = df_pred.merge(df.pivot_table(index='name', values= 'price_value_non_ref', aggfunc='mean'),how = 'inner',left_index=True, right_index=True)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'meal_type_included', aggfunc=lambda x: x.mode()),how = 'inner',left_index=True, right_index=True)
# df_pred = df_pred.merge(df.pivot_table(index='name', values= 'room_name', aggfunc= lambda x: len(x.unique())),how = 'inner',left_index=True, right_index=True)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'review_score', aggfunc='mean'),how = 'inner',left_index=True, right_index=True)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'stars', aggfunc='mean'),how = 'inner',left_index=True, right_index=True)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'room_count', aggfunc='mean'),how = 'inner',left_index=True, right_index=True)
df_pred = df_pred.merge(df.pivot_table(index='name', values= 'country', aggfunc=lambda x: x.mode()),how = 'inner',left_index=True, right_index=True)
df_pred['meal_type_included'] = np.where(df_pred['meal_type_included']== 'BREAKFAST',1,0)
df_pred['country'] = np.where(df_pred['country']== 'be',1,0)

df_pred = shuffle(df_pred,random_state = 0)
print(df_pred['is_sold_out'].value_counts())

In [ ]:
corr = df_pred.corr()# plot the heatmap
ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))
ax.set_yticklabels(
    ax.get_yticklabels(),
    rotation=0
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
prof = ProfileReport(df_pred)
prof.to_file(output_file='report.html')
prof

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

"""
Training elements
"""
context = [variable for variable in list(df_pred.columns) if variable not in ['is_sold_out']] 
# Make train/test split
x_train, x_test, y_train, y_test = train_test_split(df_pred[context], df_pred['is_sold_out'], test_size=0.2,random_state = 0,stratify = df_pred['is_sold_out'])
print(x_train.shape)
print(x_test.shape)
param_tuning = {
    'max_depth': [3, 5, 7, 10], 
    'n_estimators' : [100,200,500], 
    'max_features': ['auto', 'sqrt', 'log2'], 
    'criterion' :['gini', 'entropy'], 
    'random_state': [0]
    }

"""
Training
"""
est = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_tuning,cv = 10,n_jobs = -1,verbose = 1)

m_fit = est.fit(x_train,y_train)

"""
Predicting
"""
prob = m_fit.predict_proba(x_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, prob)
auc = roc_auc_score(y_test, prob)
fea_imp = pd.DataFrame(m_fit.best_estimator_.feature_importances_,
                       index = context,
                       columns = ['importance'])
fea_imp = fea_imp.sort_values(by = ["importance"], ascending = False)

summary = {'model':m_fit,
           'par_best_m':m_fit.best_params_,
           'auc':auc,
           'fe_importance':fea_imp}


In [ ]:
target_index = x_test.index.values
df_prob_target = pd.DataFrame(prob,target_index, columns = {"Prob_"+'is_sold_out'})
predictions = pd.concat([x_test,df_prob_target],axis = 1)

sns.set(rc={'figure.figsize':(10,6),'figure.facecolor':'white'})
sns.regplot(x=predictions['Prob_is_sold_out'], y=predictions['price_value_ref'], line_kws={"color":"r","alpha":0.9,"lw":5});

In [ ]:
ns_probs = [0 for _ in range(len(y_test))]
ns_auc = roc_auc_score(y_test, ns_probs)
rf_auc = roc_auc_score(y_test, prob)
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
dt_fpr, dt_tpr, _ = roc_curve(y_test, prob)
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Random')
plt.plot(dt_fpr, dt_tpr, marker='.', label='RF')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show();
print("AUC =", rf_auc)

In [ ]:
summary['fe_importance']
ax = sns.barplot(x=summary['fe_importance']['importance'], y=summary['fe_importance'].index)
ax.set_xlabel('Importance');

In [ ]:
sns.boxplot(data=df_pred, x="is_sold_out", y="room_count");